In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import pyodbc

In [2]:
cnxn= pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=LAPTOP-30RPIOSN;"
                      "Database=AdventureWorksDW2014;"
                      "Trusted_Connection=yes;")

In [3]:
data = pd.read_sql_query("""
select
A.[CustomerKey],
A.[ProductKey],
A.[OrderQuantity],
B.[ListPrice],
B.[EnglishProductName] as 'ProductName'
from [dbo].[FactInternetSales] as A
left join [dbo].[DimProduct] as B
on A.[ProductKey]=B.[ProductKey]
""", cnxn)

In [4]:
data.head()

,CustomerKey,ProductKey,OrderQuantity,ListPrice,ProductName
0,21768,310,1,3578.2700,"Road-150 Red, 62"
1,28389,346,1,3399.9900,"Mountain-100 Silver, 44"
2,25863,346,1,3399.9900,"Mountain-100 Silver, 44"
3,14501,336,1,699.0982,"Road-650 Black, 62"
4,11003,346,1,3399.9900,"Mountain-100 Silver, 44"


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 5 columns):
CustomerKey      60398 non-null int64
ProductKey       60398 non-null int64
OrderQuantity    60398 non-null int64
ListPrice        60398 non-null float64
ProductName      60398 non-null object
dtypes: float64(1), int64(3), object(1)
memory usage: 2.3+ MB


In [7]:
#get unique iteam
iteams=data[['ProductKey','ProductName']].drop_duplicates()

In [8]:
iteams.head()

,ProductKey,ProductName
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


In [9]:
data_q=data[['CustomerKey','ProductKey','OrderQuantity']]


In [11]:
#group together
group_data=data_q.groupby(['CustomerKey','ProductKey']).sum().reset_index()

In [13]:
group_data.head()

,CustomerKey,ProductKey,OrderQuantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [14]:
#replace a sum zero purchases with 0 and 1 to purchases
group_data.OrderQuantity.loc[group_data.OrderQuantity==0]=1

In [15]:
# getting customer with purchase
purchased= group_data.query('OrderQuantity>0')

In [16]:
purchased.head()

,CustomerKey,ProductKey,OrderQuantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [17]:
#get unique customer
customers= list(np.sort(purchased.CustomerKey.unique()))

In [18]:

 # Get our unique products that were purchased
products=list(purchased.ProductKey.unique())

In [19]:
quantity = list(purchased.OrderQuantity)


In [20]:
#get the associated rows indices cols
rows = purchased.CustomerKey.astype('category', categories = customers).cat.codes
#get the associated column indices cols
cols = purchased.ProductKey.astype('category', categories = products).cat.codes 
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
purchases_sparse
##We have 18484 customers with 158 items. For these user/item interactions, 59051 of these items had a purchase. In terms of sparsity of the matrix, that makes

<18484x158 sparse matrix of type '<class 'numpy.int32'>'
	with 59051 stored elements in Compressed Sparse Row format>

In [22]:
matrix_size=purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases=len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparcity= 100*(1-(num_purchases/matrix_size))
sparcity

97.97803231806365

# Training/validating customer/product interactions - matrix factorisation

In [23]:
# We need all customer/product interactions to find the proper matrix factorisation, so in this case we
# can't do a standard test/training split to train the model

# Instead, random products are "masked" in the original data and that will serve as a training set. 
# The test set corresponds to the original data for validation

from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://jessesw.com/images/Rec_images/MaskTrain.png")

In [24]:
# We'll use a random function to separate our training and testing sets.
import random

In [25]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [26]:
product_train,product_test,product_users_altered= make_train(purchases_sparse,pct_test = 0.2)

# Alternating Least Squares (ALS)

In [27]:
import implicit

In [33]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

100%|██████████| 50.0/50 [00:04<00:00, 12.66it/s]


In [29]:
from sklearn import metrics #Evaluating the Recommender System.  
#area under the Receiver Operating Characteristic (or ROC) curve AUC

In [30]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [ ]:
# alculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to

In [31]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.4f'%np.mean(store_auc)), float('%.4f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to four decimal places for both test and popularity benchmark

In [34]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.7724, 0.8356)

In [35]:
customers_arr=np.array(customers)  # Array of customer IDs from the ratings matrix
products_arr= np.array(products)   # array of product IDs from the rating matrix

In [ ]:
#Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [36]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return iteams.loc[iteams.ProductKey.isin(prod_codes)]

In [37]:
#Looking at the list of customers:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [38]:
get_items_purchased(11000, product_train, customers_arr, products_arr, iteams)

,ProductKey,ProductName
5432,353,"Mountain-200 Silver, 38"
5442,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain


In [39]:
#What items does the recommender system say this customer should purchase? We need to create another function that does this.
#Let's also import the MinMaxScaler from scikit-learn to help with this.
from sklearn.preprocessing import MinMaxScaler

In [41]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, iteam, num_items = 5):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    iteam- A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 5. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, iteam.ProductName.loc[iteam.ProductKey == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'ProductKey': codes, 'ProductName': descriptions}) # Create a dataframe 
    return final_frame[['ProductKey', 'ProductName']] # Switch order of columns around

In [42]:
disply_re=rec_items(11001, product_train, user_vecs, item_vecs, customers_arr, products_arr, iteams,
                       num_items = 5)
print("The top two recommended products are: \n",   
     disply_re.iloc[0,1],
     "and \n",
     disply_re.iloc[1,1])
disply_re

The top two recommended products are: 
 LL Road Tire and 
 Short-Sleeve Classic Jersey, L


,ProductKey,ProductName
0,538,LL Road Tire
1,490,"Short-Sleeve Classic Jersey, L"
2,355,"Mountain-200 Silver, 42"
3,363,"Mountain-200 Black, 46"
4,487,Hydration Pack - 70 oz.


# Recommandation

In [43]:
from IPython.display import display_html

In [44]:
def Recommandation():
    Customer_id= int(input('Customer ID:'))
    df1=(get_items_purchased(Customer_id, product_train, customers_arr, products_arr, iteams))
    df2=(rec_items(Customer_id, product_train, user_vecs, item_vecs, customers_arr, products_arr, iteams,num_items = 5))
    df1_styler = df1.style.\
                set_table_attributes("style='display:inline'").\
                set_caption('This customer has purchased:')
    df2_styler = df2.style.\
                    set_table_attributes("style='display:inline'").\
                    set_caption('This customer may like:')
    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
    
    print("So the top two recommended products are: \n",   
     df2.iloc[0,1],
     "and \n",
     df2.iloc[1,1])

In [45]:
Recommandation()

Customer ID:11000


,ProductKey,ProductName
5432,353,"Mountain-200 Silver, 38"
5442,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5476,485,Fender Set - Mountain
,ProductKey,ProductName
0,358,"Mountain-200 Black, 38"
1,467,"Half-Finger Gloves, L"
2,362,"Mountain-200 Black, 46"


So the top two recommended products are: 
 Mountain-200 Black, 38 and 
 Half-Finger Gloves, L


# Commercial Benefit

In [ ]:
#lets assume from the top 5 recommendations presented to each customer, 1 item is bought each year and costs the average 
#price of the top 5 recommendations

#we need to find the cost of the top 5 recommendations first

In [46]:
CustomerRecommandation=rec_items(11001, product_train, user_vecs, item_vecs, customers_arr, products_arr, iteams,
                       num_items = 5)

In [47]:
CustomerRecommandation

,ProductKey,ProductName
0,538,LL Road Tire
1,490,"Short-Sleeve Classic Jersey, L"
2,355,"Mountain-200 Silver, 42"
3,363,"Mountain-200 Black, 46"
4,487,Hydration Pack - 70 oz.


In [48]:
#Import Factinternetsales
prices = pd.read_sql_query("""
select
A.[ProductKey],
B.[ListPrice]
from [dbo].[FactInternetSales] as A
left join [dbo].[DimProduct] as B
on A.[ProductKey]=B.[ProductKey]
""", cnxn)

In [49]:
iteam_prices=prices[['ProductKey','ListPrice']].drop_duplicates()

In [50]:
iteam_prices.dtypes

ProductKey      int64
ListPrice     float64
dtype: object

In [51]:
CustomerRecommandation.dtypes

ProductKey      int64
ProductName    object
dtype: object

In [52]:
CustomerRecommendation=CustomerRecommandation.merge(iteam_prices,how='inner',on='ProductKey')

In [53]:
CustomerRecommendation

,ProductKey,ProductName,ListPrice
0,538,LL Road Tire,21.49
1,490,"Short-Sleeve Classic Jersey, L",53.99
2,355,"Mountain-200 Silver, 42",2319.99
3,363,"Mountain-200 Black, 46",2294.99
4,487,Hydration Pack - 70 oz.,54.99


In [57]:
print( "Based on Product list price Customer 11000's Commercial value:",'%.2f'%CustomerRecommendation.ListPrice.mean())

Based on Product list price Customer 11000's Commercial value: 949.09


In [58]:
data.CustomerKey.nunique()

18484

In [59]:
rankedcust=data.groupby('CustomerKey').agg({'OrderQuantity':'sum'}).sort_values(by='OrderQuantity', ascending=False).reset_index()

In [60]:
rankedcust.head()

,CustomerKey,OrderQuantity
0,11185,68
1,11300,67
2,11277,65
3,11262,63
4,11287,62


In [62]:
Benefit_LP=0

for i in range (1,18484):
    #top 5 recommendations for customer
    CustomerRecommendation= rec_items(rankedcust.CustomerKey[i-1], product_train, user_vecs, item_vecs, customers_arr, products_arr, iteams,num_items = 5)
    CustomerRecommendation=CustomerRecommandation.merge(iteam_prices,how='inner',on='ProductKey')
    Benefit_LP=Benefit_LP+CustomerRecommendation.ListPrice.mean()
print("Commercial Benefit (based on List Price) if all customers buy one of their top 5 recommended purchases",'%.2f'%Benefit_LP)

Commercial Benefit (based on List Price) if all customers buy one of their top 5 recommended purchases 17542030.47
